In [1]:
import pandas as pd

In [2]:
# hack to import slowkow's GTF.py (https://gist.github.com/slowkow/8101481)
# from https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path

import importlib.util
spec = importlib.util.spec_from_file_location("GTF", "/ru-auth/local/home/ezheng/ipynb/GTF.py")
GTF = importlib.util.module_from_spec(spec)
spec.loader.exec_module(GTF)

In [3]:
def process_strand(infile, reference, outfile, DNAflank):
    peaks = GTF.dataframe(filename=reference)
    peaks[0:10]

    FIMO_motifs = pd.read_table(infile)

    FIMO_motifs = FIMO_motifs[["sequence name", "start", "stop", "strand"]]
    FIMO_motifs.rename(columns={"stop":"end"}, inplace=True)
    #FIMO_motifs

    peaks.columns

    merge = pd.merge(peaks, FIMO_motifs, left_on="name", \
                     right_on="sequence name", how = 'right', suffixes=["_peak", "_motif"])

    merge[0:10]

    # do the coordinate conversion
    DNAflank = DNAflank # this is normally 200, but sometimes it is 0 -- this is the offset from the reference GTF!

    merge["start_motif_genomic"] = merge["start_peak"].astype(int) + merge["start_motif"].astype(int) - DNAflank -1 
    merge["end_motif_genomic"] = merge["start_peak"].astype(int) + merge["end_motif"].astype(int) - DNAflank -1 

    #merge.columns
    merge["dummy"] = "."
    merge[["seqname", "start_motif_genomic", "end_motif_genomic", "name", "dummy", "strand_motif"]].to_csv(\
                            outfile, \
                            header=False, index=False, sep = "\t")

In [5]:
 # Sox

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_Sox2/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/Sox_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/Sox_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

/ru-auth/local/home/ezheng/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [6]:
# Oct

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_Oct4/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/Oct_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/Oct_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

/ru-auth/local/home/ezheng/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [7]:
# tandem

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_tandem/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/tandem_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/tandem_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

/ru-auth/local/home/ezheng/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [8]:
# Sox74

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_Sox2_74/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/Sox_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/Sox74_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

# Oct28

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_Oct4_28/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/Oct_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/Oct28_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

# Oct42

infile =    "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/MEME_Oct4_42/fimo.txt"
reference = "/ru-auth/local/home/ezheng/SoxOct/chipseq/Soufi2012/Oct_peaks_200.gtf"
outfile =                            "~/SoxOct/chipseq/Soufi2012/Oct42_motifs_200.strand.bed"

process_strand(infile = infile,reference = reference,outfile = outfile, DNAflank=200)

/ru-auth/local/home/ezheng/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """
